# Creación de la gran base de datos
En este Jupyter Notebook nos encargaremos de modificar todas las bases de datos de la carpeta "datos_crudos" para transformarlas a un formato común, y así generar un archivo que contendrá toda la información que utilizaremos en el análisis final. El proceso se dividirá en los siguientes pasos:

1. Revisar los datos y su formato, para determinar si es necesario realizar alguna conversión o modificación antes de utilizarlos.
2. Corregir los datos que no sigan la estructura común, transformándolos a un formato estandarizado.
3. Unir todas las bases de datos y escribir un archivo que contenga todos los datos crudos consolidados.

In [1]:
#librerias necesarias para esta parte
import pandas as pd
import seaborn as sns
import pyarrow.parquet as pq
import pyarrow as pa
from functools import reduce

## 1. Primeramente revisaremos todos los archivos pertenecientes a la careta "datos_crudos"

### Abriremos el archivo que contiene la informacion del ingreso mediano real

In [3]:
df_imr = pd.read_excel("datos_crudos\imr general.xlsx", engine='openpyxl')
df_imr

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\2665517542.py:1: SyntaxWarning: invalid escape sequence '\i'
  df_imr = pd.read_excel("datos_crudos\imr general.xlsx", engine='openpyxl')


,NaN,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Unit of measurement,NaN,Index,NaN
2,Month,NaN,NaN,NaN
3,2018 January,NaN,NaN,109.38
4,2018 February,NaN,NaN,108.97
...,...,...,...,...
76,2024 February,NaN,NaN,158.13
77,2024 March,NaN,(p),159.19
78,Data extracted on 07 Oct 2024 23:28 UTC (GMT) ...,NaN,NaN,NaN
79,Legend:,NaN,NaN,NaN


Esta base de datos parece ser bastante mal, bamos hacerle un par de retoques

In [4]:
# existen algunas filas que tienen el nombre de las columnas, los eliminaremos ya que no son necesarias
df_imr_fix = df_imr.loc[3:77]

# al existir una columna de valor nulo, consigo las columnas para axceder mediante indexcacion
columnas_imr = list(df_imr_fix.columns)
mes_imr = pd.DataFrame({"moth": df_imr_fix[columnas_imr[0]].tolist()})

# luego de crear el dataframe nuevo que tiene la misma informacion que la columna nula, lo concateno y filtro las columnas que tienen informacion
df_imr_fix = mes_imr.merge(right= df_imr_fix, left_on="moth", right_on=columnas_imr[0])
df_imr_fix = df_imr_fix[['moth', 'Unnamed: 3']]

# por ultimo renombro la columna Unnamed: 3 ademas cambio los meses a un fromato distinto 2018 January = 2018-01
df_imr_fix = df_imr_fix.rename(columns={"moth": "mes", 'Unnamed: 3': 'ingreso mediano real'})

months_dict = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06',
                'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12'}

def funcion_remplazadora(fecha):
    ano, mes = fecha.split(" ")
    return ano + "-" + months_dict[mes]

df_imr_fix["mes"] = df_imr_fix["mes"].apply(funcion_remplazadora)

df_imr_fix.info()
df_imr_fix

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   mes                   75 non-null     object 
 1   ingreso mediano real  75 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.3+ KB


,mes,ingreso mediano real
0,2018-01,109.38
1,2018-02,108.97
2,2018-03,109.53
3,2018-04,109.84
4,2018-05,109.69
...,...,...
70,2023-11,155.08
71,2023-12,156.32
72,2024-01,158.21
73,2024-02,158.13


In [5]:
df_imr_fix.to_json(r"datos_ordenados\imr general.json", orient='records', lines=True)

### abirmos el archivo que tiene la imformacion sobre ingreso mediano real por ocupacion

In [6]:
# abrimos el archivo y vemos la info que trae
df_imr_ocupacion = pd.read_csv("datos_crudos\imr por ocupacion.csv")
df_imr_ocupacion

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\1560578292.py:2: SyntaxWarning: invalid escape sequence '\i'
  df_imr_ocupacion = pd.read_csv("datos_crudos\imr por ocupacion.csv")


,DTI_CL_UNIDAD_MEDIDA,Unit of measurement,DTI_CL_MES,Month,DTI_CL_GRUPO_OCU_IR,Occupational group,Value,Flag Codes,Flags
0,IX,Index,2018-M01,2018 January,ISCO88_1,Miembros del poder ejecutivo y de los cuerpos ...,112.57,NaN,NaN
1,IX,Index,2018-M01,2018 January,ISCO88_2,Profesionales científicos e intelectuales,107.64,NaN,NaN
2,IX,Index,2018-M01,2018 January,ISCO88_3,Técnicos y profesionales de nivel medio,109.18,NaN,NaN
3,IX,Index,2018-M01,2018 January,ISCO88_4,Empleados de oficina,108.73,NaN,NaN
4,IX,Index,2018-M01,2018 January,ISCO88_5_A,"Trabajadores de servicios personales, protecci...",111.58,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1345,12_M_VAR,12-month variation in percentages (%),2024-M03,2024 March,ISCO88_5_A,"Trabajadores de servicios personales, protecci...",9.30,p,Provisional
1346,12_M_VAR,12-month variation in percentages (%),2024-M03,2024 March,ISCO88_5_B,"Vendedores, promotores y modelos",8.50,p,Provisional
1347,12_M_VAR,12-month variation in percentages (%),2024-M03,2024 March,ISCO88_7,"Oficiales, operarios y artesanos de artes mecá...",4.90,p,Provisional
1348,12_M_VAR,12-month variation in percentages (%),2024-M03,2024 March,ISCO88_8,Operadores de instalaciones y máquinas y monta...,7.20,p,Provisional


In [7]:
# devido a que columnas que necesito son el mes, grupo de ocupacion y el valor los filtrare y tambien cambiare la columna grupo de ocupacion para
df_imr_ocupacion = df_imr_ocupacion[['Month', 'Occupational group', 'Value']]
df_imr_ocupacion['Occupational group'] = df_imr_ocupacion['Occupational group'].astype("category")

# tambien este posee el mismo sistema de la fecha entonces ocupare el mismo metodo que antes
df_imr_ocupacion.rename(columns={'Month': 'mes'}, inplace=True)
df_imr_ocupacion["mes"] = df_imr_ocupacion['mes'].apply(funcion_remplazadora)

df_imr_ocupacion.info()
df_imr_ocupacion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   mes                 1350 non-null   object  
 1   Occupational group  1350 non-null   category
 2   Value               1350 non-null   float64 
dtypes: category(1), float64(1), object(1)
memory usage: 22.9+ KB


C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\198628391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imr_ocupacion['Occupational group'] = df_imr_ocupacion['Occupational group'].astype("category")
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\198628391.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imr_ocupacion.rename(columns={'Month': 'mes'}, inplace=True)
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\198628391.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,mes,Occupational group,Value
0,2018-01,Miembros del poder ejecutivo y de los cuerpos ...,112.57
1,2018-01,Profesionales científicos e intelectuales,107.64
2,2018-01,Técnicos y profesionales de nivel medio,109.18
3,2018-01,Empleados de oficina,108.73
4,2018-01,"Trabajadores de servicios personales, protecci...",111.58
...,...,...,...
1345,2024-03,"Trabajadores de servicios personales, protecci...",9.30
1346,2024-03,"Vendedores, promotores y modelos",8.50
1347,2024-03,"Oficiales, operarios y artesanos de artes mecá...",4.90
1348,2024-03,Operadores de instalaciones y máquinas y monta...,7.20


In [8]:
df_imr_ocupacion.to_json(r"datos_ordenados\imr por categoria.json", orient='records', lines=True)

### abrimos el archivo que contiene el indice mediano real por seccion economica

In [9]:
ingorar_columnas = "A, D, F, H, J, L, N, P, R, T, V, X, Z, AB, AD, AF, AH, AJ"

df_imr_seccion = pd.read_excel("datos_crudos\imr por seccion economica.xlsx", header=3, skiprows=2,usecols=ingorar_columnas, nrows=75)

df_imr_seccion.rename(columns={'Month': 'mes'}, inplace=True)
df_imr_seccion["mes"] = df_imr_seccion['mes'].apply(funcion_remplazadora)

df_imr_seccion.info()
df_imr_seccion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   mes                                                75 non-null     object 
 1   Mining                                             75 non-null     float64
 2   Manufacturing                                      75 non-null     float64
 3   Electricity                                        75 non-null     float64
 4   Water supply                                       75 non-null     float64
 5   Construction                                       75 non-null     float64
 6   Wholesale and retail trade                         75 non-null     float64
 7   Transportation and storage                         75 non-null     float64
 8   Accommodation and food service activities          75 non-null     float64
 9   Information 

<>:3: SyntaxWarning: invalid escape sequence '\i'
<>:3: SyntaxWarning: invalid escape sequence '\i'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\151362788.py:3: SyntaxWarning: invalid escape sequence '\i'
  df_imr_seccion = pd.read_excel("datos_crudos\imr por seccion economica.xlsx", header=3, skiprows=2,usecols=ingorar_columnas, nrows=75)


,mes,Mining,Manufacturing,Electricity,Water supply,Construction,Wholesale and retail trade,Transportation and storage,Accommodation and food service activities,Information and communication,Financial and insurance activities,Real estate activities,"Professional, scientific and technical activities",Administrative and support service activities,Public administration,Education,Human health,Arts and entertainment
0,2018-01,105.82,107.48,112.61,109.26,110.41,113.96,109.55,108.67,107.17,106.21,108.73,106.34,107.46,111.54,112.71,107.82,105.61
1,2018-02,105.75,107.39,110.06,109.40,110.93,111.94,108.75,107.77,104.67,107.82,109.29,108.07,103.69,111.32,112.85,108.88,104.70
2,2018-03,105.57,107.93,112.35,114.01,110.93,112.80,107.82,109.11,106.24,107.34,110.83,108.94,106.72,111.84,112.29,110.55,105.31
3,2018-04,107.08,108.77,110.00,110.75,111.72,113.60,108.85,109.35,105.87,108.21,109.92,107.39,105.35,111.71,112.48,110.81,105.57
4,2018-05,105.03,108.61,111.30,110.72,112.88,112.52,108.60,109.03,105.99,107.42,109.17,108.24,106.15,111.34,112.78,111.01,104.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2023-11,156.55,156.38,151.87,158.75,153.95,163.78,152.20,165.85,148.61,152.76,149.24,151.62,154.26,143.79,154.53,155.88,159.43
71,2023-12,158.45,157.72,149.47,160.45,154.35,165.41,151.46,166.96,148.21,153.77,150.26,152.20,155.06,148.09,157.11,156.39,160.90
72,2024-01,161.03,160.22,151.26,162.47,158.55,166.83,153.23,167.43,151.69,153.55,153.10,153.79,155.63,149.24,158.77,159.12,164.06
73,2024-02,160.50,160.70,152.19,162.73,158.77,165.69,152.78,166.91,149.56,155.44,149.57,153.62,155.63,149.66,158.40,159.65,164.47


In [10]:
df_imr_ocupacion.to_json(r"datos_ordenados\imr por seccion.json", orient='records', lines=True)

### ahora abrimos el archivo que contiene el indice de desempleo

In [12]:
df_ind_deslab = pd.read_excel("datos_crudos\ind_deslab.xlsx", header=2)

df_ind_deslab.rename(columns={'Periodo': 'mes'}, inplace=True)
df_ind_deslab["mes"] = df_ind_deslab["mes"].astype(str)

df_ind_deslab["mes"] = df_ind_deslab["mes"].apply(lambda mes: mes[0:7])

df_ind_deslab.info()
df_ind_deslab

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   mes                                                                   80 non-null     object 
 1   1.Fuerza de trabajo ( promedio móvil trimestral, miles de personas )  80 non-null     float64
 2   2.Empleo ( promedio móvil trimestral, miles de personas )             80 non-null     float64
 3   3.Tasa  de  desempleo  (porcentaje)                                   80 non-null     float64
dtypes: float64(3), object(1)
memory usage: 2.6+ KB


<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\1622802872.py:1: SyntaxWarning: invalid escape sequence '\i'
  df_ind_deslab = pd.read_excel("datos_crudos\ind_deslab.xlsx", header=2)


,mes,"1.Fuerza de trabajo ( promedio móvil trimestral, miles de personas )","2.Empleo ( promedio móvil trimestral, miles de personas )",3.Tasa de desempleo (porcentaje)
0,2018-01,9438.115290,8793.922635,6.825437
1,2018-02,9449.494321,8787.079741,7.010053
2,2018-03,9456.608476,8759.080753,7.376088
3,2018-04,9441.395486,8781.083740,6.993794
4,2018-05,9465.329449,8766.651220,7.381446
...,...,...,...,...
75,2024-04,10209.534117,9339.295511,8.523784
76,2024-05,10177.707624,9331.646959,8.312880
77,2024-06,10170.424862,9321.379688,8.348178
78,2024-07,10137.870129,9257.479667,8.684176


In [13]:
df_imr_ocupacion.to_json(r"datos_ordenados\ind_deslab.json", orient='records', lines=True)

### ahora abrimos el archivo que contiene el indice de precios al consumidor

In [14]:
df_ipc = pd.read_excel("datos_crudos\ipc.xlsx", header=2)

df_ipc.rename(columns={'Periodo': 'mes', "1.Índice IPC General": "indice de precios consumidor"}, inplace=True)
df_ipc["mes"] = df_ipc["mes"].astype(str)

df_ipc["mes"] = df_ipc["mes"].apply(lambda mes: mes[0:7])

df_ipc.info()
df_ipc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   mes                           81 non-null     object 
 1   indice de precios consumidor  81 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.4+ KB


<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\2373417067.py:1: SyntaxWarning: invalid escape sequence '\i'
  df_ipc = pd.read_excel("datos_crudos\ipc.xlsx", header=2)


,mes,indice de precios consumidor
0,2018-01,74.27
1,2018-02,74.31
2,2018-03,74.46
3,2018-04,74.70
4,2018-05,74.90
...,...,...
76,2024-05,103.52
77,2024-06,103.42
78,2024-07,104.19
79,2024-08,104.45


In [15]:
df_imr_ocupacion.to_json(r"datos_ordenados\ipc.json", orient='records', lines=True)

### abrimos el archivo que contiene la informacion sobre el indice de Precios Selectivo de Acciones

In [16]:
df_ipsa = pd.read_excel("datos_crudos\ipsa.xlsx", header=2)

df_ipsa.rename(columns={'Periodo': 'mes', '1.IPSA  (índice enero 2003=1000)                      ': "indice de Precios Selectivo de Acciones"}, inplace=True)
df_ipsa["mes"] = df_ipsa["mes"].astype(str)

df_ipsa["mes"] = df_ipsa["mes"].apply(lambda mes: mes[0:7])

df_ipsa.info()
df_ipsa

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 2 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   mes                                      81 non-null     object 
 1   indice de Precios Selectivo de Acciones  81 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.4+ KB


<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\2268243654.py:1: SyntaxWarning: invalid escape sequence '\i'
  df_ipsa = pd.read_excel("datos_crudos\ipsa.xlsx", header=2)


,mes,indice de Precios Selectivo de Acciones
0,2018-01,5855.38
1,2018-02,5602.83
2,2018-03,5542.22
3,2018-04,5710.90
4,2018-05,5455.09
...,...,...
76,2024-05,6632.80
77,2024-06,6413.88
78,2024-07,6440.56
79,2024-08,6459.96


In [17]:
df_ipsa.to_json(r"datos_ordenados\ipsa.json", orient='records', lines=True)

### abrimos el archivo que contiene toda la infromacion sobre el producto interno bruto

In [18]:
df_pib = pd.read_excel("datos_crudos\pib (trimestral).xlsx", header=2)

df_pib.rename(columns={'Periodo': 'mes'}, inplace=True)
df_pib["mes"] = df_pib["mes"].astype(str)

df_pib["mes"] = df_pib["mes"].apply(lambda mes: mes[0:7])

df_pib_tercero = df_pib.copy()
df_pib_segundo = df_pib.copy()

def menos_1_mes(fecha):
    ano, mes = fecha.split("-")
    return ano + "-" + str(int(mes)-1).zfill(2)

df_pib_segundo["mes"] = df_pib_segundo["mes"].apply(menos_1_mes)

df_pib_primero = df_pib_segundo.copy()
df_pib_primero["mes"] = df_pib_primero["mes"].apply(menos_1_mes)

def ordenar(fecha_series):
    return fecha_series.apply(lambda fecha: tuple(map(int, fecha.split('-'))))

df_pib_complete = pd.concat([df_pib_tercero, df_pib_segundo, df_pib_primero], ignore_index=True)
df_pib_complete = df_pib_complete.sort_values(by="mes", key=ordenar).reset_index(drop=True)
df_pib_complete

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\2354665543.py:1: SyntaxWarning: invalid escape sequence '\p'
  df_pib = pd.read_excel("datos_crudos\pib (trimestral).xlsx", header=2)


,mes,1.PIB a precios corrientes,2.PIB volumen a precios del año anterior encadenado,3.PIB volumen a precios del año anterior encadenado (desestacionalizado)
0,2018-01,46986.271371,46511.870532,46837.401890
1,2018-02,46986.271371,46511.870532,46837.401890
2,2018-03,46986.271371,46511.870532,46837.401890
3,2018-04,47033.691982,47707.607554,47461.533987
4,2018-05,47033.691982,47707.607554,47461.533987
...,...,...,...,...
67,2023-08,68330.323630,49625.097412,51057.807019
68,2023-09,68330.323630,49625.097412,51057.807019
69,2023-10,74492.255823,53284.210836,51034.247215
70,2023-11,74492.255823,53284.210836,51034.247215


In [19]:
df_imr_ocupacion.to_json(r"datos_ordenados\pib.json", orient='records', lines=True)

### y ahora creamos el dataframe definitivo

In [20]:
lista_df = [df_pib_complete, df_imr_fix, df_imr_seccion, df_ind_deslab, df_ipc, df_ipsa]

for i, df in enumerate(lista_df):
    df['mes'] = pd.to_datetime(df['mes'])
    lista_df[i] = df[df["mes"] <= "2024-06-30"]

df_economia = reduce(lambda left, right: pd.merge(left, right, on="mes", how="outer"), lista_df)

nombre_columnas_df_economia = [
    "fecha",
    "PIB_precio_corriente",
    "PIB_volumen_encadenado",
    "PIB_volumen_desestacionalizado",
    "ingreso_mediano_real",
    "mineria",
    "manufactura",
    "electricidad",
    "suministro_agua",
    "construccion",
    "comercio_mayorista_minorista",
    "transporte_almacenaje",
    "alojamiento_alimentos",
    "informacion_comunicacion",
    "finanzas_seguros",
    "bienes_raices",
    "servicios_cientificos_tecnicos",
    "servicios_administrativos_apoyo",
    "administracion_publica",
    "educacion",
    "salud_humana",
    "arte_entretenimiento",
    "fuerza_laboral_trimestral",
    "empleo_trimestral",
    "tasa_desempleo",
    "IPC",
    "IPSA"
]

df_economia.columns = nombre_columnas_df_economia

df_economia.to_parquet('DataFrames_economia.parquet')

# Datadrame pib regional

In [22]:
df_pib_reg = pd.read_excel("datos_crudos\excel_data_20241118115318.xlsx", header=5)
df_pib_reg.dropna(axis=0, inplace=True)
df_pib_reg.columns = [
    "fecha", "norte", "centro", "sur", "austral", "centro sur",
    "region metropolitana", "Extraregional", "PIB sumado (chile en general)"]

df_pib_reg = df_pib_reg[df_pib_reg["fecha"] >= "2018-01-01"]
df_pib_reg

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_16192\4109335562.py:1: SyntaxWarning: invalid escape sequence '\e'
  df_pib_reg = pd.read_excel("datos_crudos\excel_data_20241118115318.xlsx", header=5)


,fecha,norte,centro,sur,austral,centro sur,region metropolitana,Extraregional,PIB sumado (chile en general)
17,2018-01-01,1.449937,0.075026,0.358432,0.007927,0.917920,1.368707,0.336995,4.514944
18,2018-04-01,0.892162,0.391718,0.498601,0.028064,1.335233,1.787266,0.577394,5.510437
19,2018-07-01,0.038885,0.139813,0.343651,0.031508,0.612539,0.973988,0.286388,2.426772
20,2018-10-01,0.533998,0.166607,0.384401,-0.033418,0.805334,1.422219,0.232423,3.511565
21,2019-01-01,-0.472847,0.018551,0.249236,0.083904,-0.037534,1.162557,0.195670,1.199536
22,2019-04-01,-0.168100,0.066426,0.105399,0.073621,0.089795,0.957005,0.071726,1.195873
23,2019-07-01,0.803486,0.345910,0.321217,0.035504,0.328777,1.197201,0.149334,3.181430
24,2019-10-01,-0.144830,-0.234544,-0.129794,0.030043,-0.222579,-1.601884,-0.425382,-2.728971
25,2020-01-01,0.707111,0.032258,0.021664,-0.036711,-0.240715,-0.598083,-0.119288,-0.233765
26,2020-04-01,-0.524242,-1.305588,-0.870697,-0.264530,-2.010270,-7.903447,-2.073959,-14.952734


In [24]:
df_pib_reg.to_parquet("PIB_regional.parquet")

In [2]:
df_pib_reg = pd.read_excel("Datos_crudos_1\PBI_miles de millones de pesos encadenados.xlsx", header=2)
df_pib_reg.head()

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\jpose75\AppData\Local\Temp\ipykernel_8136\2112835970.py:1: SyntaxWarning: invalid escape sequence '\P'
  df_pib_reg = pd.read_excel("Datos_crudos_1\PBI_miles de millones de pesos encadenados.xlsx", header=2)


,Reg,Descripción series,2013-03-01 00:00:00,2013-06-01 00:00:00,2013-09-01 00:00:00,2013-12-01 00:00:00,2014-03-01 00:00:00,2014-06-01 00:00:00,2014-09-01 00:00:00,2014-12-01 00:00:00,...,2022-06-01 00:00:00,2022-09-01 00:00:00,2022-12-01 00:00:00,2023-03-01 00:00:00,2023-06-01 00:00:00,2023-09-01 00:00:00,2023-12-01 00:00:00,2024-03-01 00:00:00,2024-06-01 00:00:00,2024-09-01 00:00:00
0,1,Región de Arica y Parinacota,278.304651,306.132341,338.475536,321.351073,285.116735,330.556785,325.580361,336.348957,...,419.498068,418.237390,432.919088,361.423244,392.463569,402.110757,441.328576,372.418846,403.393797,NaN
1,2,Región de Tarapacá,811.931803,926.814620,1044.963002,1135.992471,977.620431,990.721233,954.180471,1087.410974,...,1206.877580,1186.946165,1218.831306,1172.444308,1155.646190,1270.080071,1345.905667,1288.441778,1261.026700,NaN
2,3,Región de Antofagasta,3759.318361,3949.000465,3792.362453,4133.677309,3803.705319,4204.009406,3887.046755,4338.462348,...,3840.022305,3908.559895,4177.589301,3806.672045,3836.066866,4079.008357,4225.898588,4102.717117,4184.394867,NaN
3,4,Región de Atacama,871.758026,845.358870,867.192979,975.218096,905.985208,868.805299,939.175426,982.605582,...,980.557661,955.214395,1069.101321,983.429140,1026.342383,1014.970777,1070.271071,1011.940845,1000.007554,NaN
4,5,Región de Coquimbo,1426.567416,1413.417680,1341.707470,1436.152081,1386.119006,1359.612421,1264.773505,1393.829271,...,1497.901162,1485.615197,1642.527817,1567.639415,1565.762918,1548.771544,1637.243366,1600.553436,1607.844404,NaN


In [3]:
df_pib_reg = df_pib_reg.T
new_columns = df_pib_reg.iloc[1]
df_pib_reg = df_pib_reg[2:]
df_pib_reg.columns = new_columns
df_pib_reg = df_pib_reg.dropna(axis=0).drop(columns="Producto Interno Bruto").reset_index()

df_pib_reg.head()

Descripción series,index,Región de Arica y Parinacota,Región de Tarapacá,Región de Antofagasta,Región de Atacama,Región de Coquimbo,Región de Valparaíso,Región Metropolitana de Santiago,Región del Libertador General Bernardo OHiggins,Región del Maule,Región de Ñuble,Región del Biobío,Región de La Araucanía,Región de Los Ríos,Región de Los Lagos,Región de Aysén del General Carlos Ibáñez del Campo,Región de Magallanes y de la Antártica Chilena,Subtotal regionalizado,Extrarregional
0,2013-03-01,278.304651,811.931803,3759.318361,871.758026,1426.567416,3339.833326,16843.137386,1931.429678,1599.502893,634.644423,2301.626148,1172.639779,523.442501,1208.470375,282.046799,387.047791,37312.836445,4008.301783
1,2013-06-01,306.132341,926.81462,3949.000465,845.35887,1413.41768,3371.955157,17862.986368,1708.125163,1610.473179,650.65431,2475.065928,1068.709189,520.977785,1237.106367,272.044536,386.397966,38555.531243,4179.923592
2,2013-09-01,338.475536,1044.963002,3792.362453,867.192979,1341.70747,3190.091221,17184.74542,1504.860461,1285.815102,553.334016,2404.948751,1006.802439,492.418296,1147.297226,265.659402,379.493237,36778.189428,4226.528054
3,2013-12-01,321.351073,1135.992471,4133.677309,975.218096,1436.152081,3438.771257,18822.236849,1694.010822,1467.454453,599.951505,2532.224807,1084.623743,533.889209,1311.711077,279.318343,431.384039,40173.157448,4633.475802
4,2014-03-01,285.116735,977.620431,3803.705319,905.985208,1386.119006,3271.999963,17350.239851,1934.947837,1583.320053,627.795167,2394.915803,1175.242401,543.00567,1279.226153,276.013977,435.493723,38178.287411,4153.708989


In [4]:
df_pib_reg['index'] = pd.to_datetime(df_pib_reg['index'])
df_pib_reg.rename(columns={'index': "fecha"}, inplace=True)
df_pib_reg.columns.name = None
df_pib_reg.head()

,fecha,Región de Arica y Parinacota,Región de Tarapacá,Región de Antofagasta,Región de Atacama,Región de Coquimbo,Región de Valparaíso,Región Metropolitana de Santiago,Región del Libertador General Bernardo OHiggins,Región del Maule,Región de Ñuble,Región del Biobío,Región de La Araucanía,Región de Los Ríos,Región de Los Lagos,Región de Aysén del General Carlos Ibáñez del Campo,Región de Magallanes y de la Antártica Chilena,Subtotal regionalizado,Extrarregional
0,2013-03-01,278.304651,811.931803,3759.318361,871.758026,1426.567416,3339.833326,16843.137386,1931.429678,1599.502893,634.644423,2301.626148,1172.639779,523.442501,1208.470375,282.046799,387.047791,37312.836445,4008.301783
1,2013-06-01,306.132341,926.81462,3949.000465,845.35887,1413.41768,3371.955157,17862.986368,1708.125163,1610.473179,650.65431,2475.065928,1068.709189,520.977785,1237.106367,272.044536,386.397966,38555.531243,4179.923592
2,2013-09-01,338.475536,1044.963002,3792.362453,867.192979,1341.70747,3190.091221,17184.74542,1504.860461,1285.815102,553.334016,2404.948751,1006.802439,492.418296,1147.297226,265.659402,379.493237,36778.189428,4226.528054
3,2013-12-01,321.351073,1135.992471,4133.677309,975.218096,1436.152081,3438.771257,18822.236849,1694.010822,1467.454453,599.951505,2532.224807,1084.623743,533.889209,1311.711077,279.318343,431.384039,40173.157448,4633.475802
4,2014-03-01,285.116735,977.620431,3803.705319,905.985208,1386.119006,3271.999963,17350.239851,1934.947837,1583.320053,627.795167,2394.915803,1175.242401,543.00567,1279.226153,276.013977,435.493723,38178.287411,4153.708989


In [48]:
conversion_regiones = {
    'Región de Arica y Parinacota\xa0': 'Arica',
    'Región de Tarapacá': 'Tarapaca',
    'Región de Antofagasta': 'Antofagasta',
    'Región de Atacama': 'Atacama',
    'Región de Coquimbo': 'Coquimbo',
    'Región de Valparaíso': 'Valparaiso',
    'Región Metropolitana de Santiago': 'Metropolitana',
    'Región del Libertador General Bernardo OHiggins': 'Libertador',
    'Región del Maule': 'Maule',
    'Región de Ñuble': 'Nuble',
    'Región del Biobío': 'Biobio',
    'Región de La Araucanía': 'Araucania',
    'Región de Los Ríos': 'Losrios',
    'Región de Los Lagos': 'Loslagos',
    'Región de Aysén del General Carlos Ibáñez del Campo': 'Aysén',
    'Región de Magallanes y de la Antártica Chilena': 'Magallanes'
}

In [49]:
repetir = len(list(df_pib_reg.columns))
columnas = list(df_pib_reg.columns)
df_fecha = df_pib_reg.iloc[:, 0].reset_index(drop=True)
largo = len(df_fecha)

df_pib_complete = pd.DataFrame()

for i in range(1, repetir-2):
    df_pib_reg_column_ac = df_pib_reg.iloc[:, i].reset_index()
    df_pib_reg_column_ac = df_pib_reg_column_ac.rename(columns={columnas[i]: "PIB"}).drop(columns="index")
    df_pib_reg_column = pd.DataFrame({"region": [columnas[i]] * largo})
    df_pib_reg_column1 = pd.concat([df_pib_reg_column_ac, df_pib_reg_column], axis=1)
    df_pib_reg_column2 = pd.concat([df_fecha, df_pib_reg_column1], axis=1)
    if df_pib_complete.empty:
        df_pib_complete = df_pib_reg_column2
    else:
        df_pib_complete = pd.concat([df_pib_complete, df_pib_reg_column2], axis=0)

df_pib_complete["PIB"] = df_pib_complete["PIB"].astype(float)
df_pib_complete['region'] = df_pib_complete['region'].apply(lambda x: conversion_regiones.get(x, x))
df_pib_complete

,fecha,PIB,region
0,2013-03-01,278.304651,Arica
1,2013-06-01,306.132341,Arica
2,2013-09-01,338.475536,Arica
3,2013-12-01,321.351073,Arica
4,2014-03-01,285.116735,Arica
...,...,...,...
41,2023-06-01,468.905582,Magallanes
42,2023-09-01,442.940103,Magallanes
43,2023-12-01,496.159006,Magallanes
44,2024-03-01,498.067054,Magallanes


In [50]:
df_pib_complete.to_parquet("Datos_ordenados_1/pib_regional.parquet")